### Dice trading

##### 4.a)

$S_t = \{p_t, d_t\}$, avec $p_t$ le nombre de points au tour t et $d_t$ le nombre de dés au tour t

$a_t = \{0, 1\}$, soit acheter un nouveau dé, soit ne rien faire au tour t

$p_t \geq 0$

$d_t \leq 3$

$d_{t+1} = d_t + a_{d+1}$

$p_{t+1} = p_t - 5*a_t + \max_{i \in \{1, d_t\}} (u_i (1, 2, 3, 4, 5, 6))$, le nombre de points est déterminé par le score max de tous les dés

##### 4.b) Implement an heuristic

3 variables décrivent l'état du système S :
- Le total de point $p_{t}$. Ce total commençant à 0, et le gain maximum étant de 6 par tour, on en déduit l'encadrement suivant : $0 \leq p \leq 60$
- L'action d'acheter un dé $a_{t}$. On a $a_t \in {0,1}$
- Le tour $t$, qui est compris entre 1 et 10

On en déduit que toute stratégie peut être représentée par une matrice 3D de dimension $61*3*10$

REMARQUE IMPORTANTE POUR LA SUITE : La représentation d'une heuristique par une matrice n'est pas unique, car certains états sont inaccessibles (par exemple, avoir plus de 0 points au premier tour). Cela sera pertinent pour la question 4.g).

In [81]:
T=10
never_buy_heuristic = zeros(6*T + 1, 3, T)


buy_heuristic = ones(6*T + 1, 3, T)
for p in 1:6*T + 1
    for t in 1:T
        buy_heuristic[p, 3, t] = 0 # can't buy more than 3 dices
    end
end
for p in 1:6
    for t in 1:T
        for d in 1:3
            buy_heuristic[p, d, t] = 0 # can't buy dices with less than 6 points
        end
    end
end

In [82]:
#=  OBSOLETE CODE
function never_buy_heuristic(s,t) # Always returns "Don't Buy" (never buy dice) 
    return "Don't Buy"
end
=#

##### 4.c) Write a simulator giving $V_{\pi}$ in a confidence interval of 95%

In [83]:
using Pkg
Pkg.add("Distributions")
using Distributions
using Random

   Resolving package versions...
  No Changes to `C:\Users\erwan\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\erwan\.julia\environments\v1.8\Manifest.toml`


In [84]:
#= OBSOLETE CODE
function simulator_simple_game(heuristic, history=false, T=10)
    nb_dice = 1
    total_points = 0
    println("test")
    if history
        history_list = []
    end

    for turn in 1:T
        action = heuristic((total_points, nb_dice), turn)
        @assert action != "Buy" || total_points >= 6 "RuleError You don't have enough points to buy a dice"
        if action == "Buy" && total_points >= 6 # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
            nb_dice += 1
            total_points -= 5
        end
        dice_roll = maximum(rand(1:6, nb_dice))
        total_points += dice_roll

        if history
            push!(history_list, (action, dice_roll, total_points))
        end
    end
    
    if history
        return total_points, history_list
    end
    return total_points
end
simulator_simple_game(never_buy_heuristic, true, 10)
=#

In [85]:
function simulator_simple_game(politic, T=10, N= 100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            action = politic[total_points + 1, nb_dice, turn] # Due to Julia's indexing system, we need to add 1 to the total_points
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, turn))"
            @assert action != 1 || nb_dice < 3       "RuleError You can't have more than 3 dices. State: $((total_points, nb_dice, turn))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += dice_roll
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
end

T=10
simulator_simple_game(buy_heuristic, T, 10000)

(35.8418, [35.74596108307539, 35.937638916924605])

In [86]:
# function compute_expected_cost(heuristic, p=0.05):
#     #p is the error level
#     n=0 # Number of game simulated
#     emp_mean=0
#     emp_dev=0

#     d = Normal()  # Crée une distribution normale standard
#     p = 0.95  # Probabilité pour laquelle vous voulez trouver le quantile
#     q = quantile(d, p)  # Calcule le quantile
    
#     while n<1 and 
#     end
# end

##### 4.d)

On note $X$, $Y$, $Z$ les variables aléatoires associées aux trois dés. Par indépendance, on a :

$\forall k \in 1:6$

$P(Y=k)=1/6$

$P(max(X,Y)\leq k) = P((X\leq k) \cap (Y\leq k)) = P(X\leq k)*P(Y\leq k) = \frac{k^2}{36} \Rightarrow P(max(X,Y)=k) = P(max(X,Y)\leq k) - P(max(X,Y)\leq k-1) = \frac{2k-1}{36}$

$P(max(X,Y,Z)\leq k) = P((X\leq k) \cap (Y\leq k) \cap (Z\leq k)) = P(X\leq k)*P(Y\leq k)*P(Z\leq k) = \frac{k^3}{216} \Rightarrow P(max(X,Y,Z)=k) = P(max(X,Y,Z)\leq k) - P(max(X,Y,Z)\leq k-1) = \frac{3k^2-3k+1}{216}$

In [87]:
#= ??? Useless code ???
for i in 1:6
    println(1 * (i)^2 * 3 - 2)
end
=#

In [99]:
function law_max(nb_dices)
    @assert nb_dices > 0 "RuleError You must have at least one dice"
    @assert nb_dices <= 3 "RuleError You can't have more than 3 dices"
    
    if nb_dices == 1
        return DiscreteUniform(1, 6)
    end
    
    if nb_dices == 2
        values = 1:6
        probs = [1/36, 3/36, 5/36, 7/36, 9/36, 11/36]
        return Categorical(probs)
    end

    if nb_dices == 3
        values = 1:6
        probs = [1/216, 7/216, 19/216, 37/216, 61/216, 91/216]
        return Categorical(probs)
    end
end

function general_law_max(nb_dices) #useful for 4.h)
    @assert nb_dices > 0 "RuleError You must have at least one dice"
    if nb_dices == 1
        return DiscreteUniform(1, 6)
    end
    values = 1:6
    probs = [(k/6)^nb_dices - ((k-1)/6)^nb_dices for k in values]
    return Categorical(probs)
end

general_law_max (generic function with 1 method)

##### 4.e)

In [89]:
#= Outdated code ???
Q = zeros(61, 3, 10, 2) #60 points, 3 dices, 10 turns, 2 actions

#Construct the value function for each state at each time
V = zeros(61, 3, 10)

P = zeros(61, 3, 10, 2, 61) #60 points, 3 dices, 10 turns, 2 actions, 60 points 

for p in 0:60
    for d in 1:3
        for t in 1:10
            for a in 0:1
                if d + a <= 3
                    law = law_max(d + a)
                    for p_prime in 0:60
                        P[p + 1, d, t, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                    end
                end
            end
        end
    end
end
policy_markov = zeros(61, 3, 10)

println(law_max(2)[5])
println(pdf(law_max(2), 5))
=#

We now proceed to the dynamic programming

In [90]:
function dp(T)
    Q = zeros(T*6 + 1, 3, T, 2) # Action-value function Q(s,a), giving the estimates the expected cumulative future rewards starting from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions)
    #Construct the value function for each state at each time
    V = zeros(T*6 + 1, 3, T+1) # Value function V(s), giving the expected cumulative future rewards starting from state s (60 points, 3 dices, 10 turns + 1 for the final state)

    P = zeros(T*6 + 1, 3,  2, T*6 + 1) # Transition probability matrix P(s'|s,a), giving the probability of transitioning to state s' from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions, 60 points)
    # Note that s = (p,d,t), but the probability doesn't depend on t nor d, so we can simplify the matrix to P(p'|p,a)
    for p in 0:T*6
        for d in 1:3
            for a in 0:1
                if d + a <= 3
                    law = law_max(d + a) # law_max returns the distribution of the maximum of d + a dices, pdf(law, k) gives the probability of rolling k
                    for p_prime in 0:T*6
                        P[p + 1, d, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                    end
                end
            end
        end
    end
    policy_markov = zeros(T * 6 + 1, 3, T)
    for t in T:-1:1
        for p in 0:T * 6
            for d in 1:3
                V[p + 1, d, t] = -100000 # We initialize the value function to a very low value
                for a in 0:1
                    Q[p + 1, d, t, a + 1] = 0
                    if d + a <= 3 && a * (p-6) >= 0        # We assure that we can't have more than 3 dices AND that we can't buy a dice if we don't have enough points
                        for p_prime in 0:T * 6
                            transition_cost = p_prime - p
                            Q[p + 1, d, t, a + 1] += P[p + 1, d, a + 1, p_prime + 1] * (transition_cost + V[p_prime + 1, d + a, t+1])
                        end
                        if Q[p + 1, d, t, a + 1] > V[p + 1, d, t]
                            V[p + 1, d, t] = Q[p + 1, d, t, a + 1]
                            policy_markov[p + 1, d, t] = a
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end

dp (generic function with 1 method)

### La valeur optimale trouvée par programmation dynamique est la suivante

In [91]:
T=10
policy_markov = dp(T)
C = simulator_simple_game(policy_markov, T, 10000)

UndefVarError: UndefVarError: True not defined

 #### On est content car elle est mieux que la valeur de l'heuristique

##### 4.g)
Comme indiqué en 4.b), la stratégie "ne jamais acheté" peut être représentée par autre chose qu'une matrice nulle (à cause des valeurs inaccesibles). Ces valeurs inaccesibles sont  :

- $\pi [p,1,t]$ avec $p>6(t-1)$ ou $p<t-1$

- $\pi [p,2,t]$ avec $t=1$ ou $p>6(t-1) - 5$ ou $p<t-1-5$

- $\pi [p,3,t]$ avec $t<=2$ ou $p>6(t-1) - 10$ ou $p<t-1-10$

Qu'on peut résumer en :

- $\pi [p,d,t]$ avec $d+1>t$ ou $p>6(t-1) - 5*(d-1)$ ou $p<t - 1 - 5(d-1)$

(On peut remarquer que la formule précédente marche aussi pour $d$ différent de 3)

Pour résoudre ce problème, la fonction suivante mettra toutes ces valeurs inaccessible à 0.


In [92]:
function nullify_inaccessible_values(matrice_3d)
    for p in 1:size(matrice_3d, 1)
        for d in 1:size(matrice_3d, 2)
            for t in 1:size(matrice_3d, 3)
                if d+1>t || p-1 > 6*(t-1) - 5*(d-1) || p - 1 < t - 1 - 5*(d - 1) # don't forget that Julia's indexing system starts at 1
                    matrice_3d[p, d, t] = 0
                end
            end
        end
    end
    return matrice_3d
end

nullify_inaccessible_values (generic function with 1 method)

In [100]:
# We will now implement the dynamic programming with descending time horizon to see when it is never intersting to buy a dice
function is_null(matrice_3d)
    for i in 1:size(matrice_3d, 1)
        for j in 1:size(matrice_3d, 2)
            for k in 1:size(matrice_3d, 3)
                if matrice_3d[i, j, k] != 0
                    return false
                end
            end
        end
    end
    return true
end

for T in 10:-1:1
    policy_markov = nullify_inaccessible_values(dp(T))
    if is_null(policy_markov)
        println("The optimal policy is to never buy a dice after $T turns")
        break
    end
end

The optimal policy is to never buy a dice after 6 turns


In [110]:
function simple_test_strategy(T, π, d=1)
    for t in 1:T
        println("t=$t : ", π[:, d, t])
    end
end
simple_test_strategy(10, nullify_inaccessible_values(dp(10)))

t=1 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=2 : [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=3 : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=4 : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=5 : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=6 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### 4.h)
On reprend les fonctions précédentes en enlevant la condition d<3 :

In [106]:
function no_dice_limit_simulator(politic, T=10, N= 100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    Liste = [] # List of the total points after T turns for each game
    for j in 1:N
        total_points = 0 # The player starts with 0 points
        nb_dice = 1 # The player starts with 1 dice
        for turn in 1:T
            action = politic[total_points + 1, nb_dice, turn] # Due to Julia's indexing system, we need to add 1 to the total_points
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice. State: $((total_points, nb_dice, turn))"
            if action == 1  # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += dice_roll
        end
        push!(Liste, total_points) 
    end
    return mean(Liste), [mean(Liste) - (1.96*std(Liste)/sqrt(N)), mean(Liste) + (1.96*std(Liste)/sqrt(N))]
end

function no_dice_limit_dp(T)
    Q = zeros(T*6 + 1, T+1, T, 2) # Action-value function Q(s,a), giving the estimates the expected cumulative future rewards starting from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions)
    #Construct the value function for each state at each time
    V = zeros(T*6 + 1, T+1, T+1) # Value function V(s), giving the expected cumulative future rewards starting from state s (60 points, 3 dices, 10 turns + 1 for the final state)

    P = zeros(T*6 + 1, T+1,  2, T*6 + 1) # Transition probability matrix P(s'|s,a), giving the probability of transitioning to state s' from state s, taking action a (60 points, 3 dices, 10 turns, 2 actions, 60 points)
    # Note that s = (p,d,t), but the probability doesn't depend on t nor d, so we can simplify the matrix to P(p'|p,a)
    for p in 0:T*6
        for d in 1:T
            for a in 0:1
                law = general_law_max(d + a) # law_max returns the distribution of the maximum of d + a dices, pdf(law, k) gives the probability of rolling k
                for p_prime in 0:T*6
                    P[p + 1, d, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                end
            end
        end
    end
    policy_markov = zeros(T * 6 + 1, T+1, T)
    for t in T:-1:1
        for p in 0:T * 6
            for d in 1:T
                V[p + 1, d, t] = -100000 # We initialize the value function to a very low value
                for a in 0:1
                    Q[p + 1, d, t, a + 1] = 0
                    if  a * (p-6) >= 0        # We assure that we can't buy a dice if we don't have enough points
                        for p_prime in 0:T * 6
                            transition_cost = p_prime - p
                            Q[p + 1, d, t, a + 1] += P[p + 1, d, a + 1, p_prime + 1] * (transition_cost + V[p_prime + 1, d + a, t+1])
                        end
                        if Q[p + 1, d, t, a + 1] > V[p + 1, d, t]
                            V[p + 1, d, t] = Q[p + 1, d, t, a + 1]
                            policy_markov[p + 1, d, t] = a
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end

no_dice_limit_dp (generic function with 1 method)

Remarquons que dans le cas $T=10$, pas plus qu'un dé n'était acheté. Considérons le cas $T=50$

In [108]:
simple_test_strategy(50, nullify_inaccessible_values(no_dice_limit_dp(50)), 5)

t=1 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
t=2 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

On remarque qu'il reste interessant d'acheter des dés au delà du 5ème dé dans certaines situations.